In [1]:
import numpy as np
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
from craft import uvfits, craco_plan, fdmt as FDMT
import craco.search_pipeline
from craco.pyxrtutil import pyxrt

In [3]:
xclbin='/data/craco/ban115/builds/binary_container_fe3fef0d.xclbin'
alloc_device_only = True

fits_file = "/home/gup037/tmp/frb_d0_t0_a1_sninf_lm00.fits"
f = uvfits.open(fits_file)
values = craco.search_pipeline.get_parser().parse_args([])
values.uv = fits_file

In [4]:
device = pyxrt.device(0)
xbin = pyxrt.xclbin(xclbin)
uuid = device.load_xclbin(xbin)

In [5]:
plan = craco_plan.PipelinePlan(f, values)

/home/gup037/Codes/craft/src/craft/uvfits.py:100: UserWarning: Unknown int time in file. returning 1ms
  warnings.warn('Unknown int time in file. returning 1ms')
/home/gup037/Codes/craft/src/craft/craco_plan.py:140: UserWarning: Cannot grid things on U=0 or V=0 blid=515.0 2-3 uvpix=(254, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:140: UserWarning: Cannot grid things on U=0 or V=0 blid=515.0 2-3 uvpix=(253, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:140: UserWarning: Cannot grid things on U=0 or V=0 blid=260.0 1-4 uvpix=(253, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:140: UserWarning: Cannot grid things on U=0 or V=0 blid=1029.0 4-5 uvpix=(0, 251)
  warnings.warn(f'Cannot grid things on U=0 or V=0

Upper registers [(3, 1), (4, 1), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2), (40, 2), (41, 2), (42, 2), (43, 2), (44, 2), (45, 2), (46, 2), (87, 2), (88, 2), (89, 2), (90, 2), (91, 2), (92, 2), (93, 2), (94, 2), (95, 2), (96, 2), (97, 2), (98, 2), (99, 2), (100, 2), (101, 2), (102, 2), (103, 2)]
Lower registers [(1, 3), (1, 4), (2, 4), (3, 4), (3, 5), (3, 6), (4, 6), (4, 7), (2, 8), (4, 8), (2, 9), (3, 9), (4, 9), (5, 9), (3, 10), (5, 10), (7, 10), (3, 11), (5, 11), (6, 11), (7, 11), (7, 12), (8, 12), (8, 13), (9, 13), (7, 14), (8, 14), (9, 14), (8, 15), (8, 16), (9, 16), (2, 17)]


In [6]:
nt = 256
fake_block = np.ones((plan.nbl, plan.nf, nt))

In [7]:
fake_block.shape

(190, 256, 256)

In [8]:
f2 = uvfits.open("/data/craco/gup037/test_runs_of_craco_pipeline/pretty_pictures/Solar_eclipse_testing_obs/SB049168/SB049168.b00_beam00.uvfits")

In [14]:
data_source = f.time_blocks_with_uvws(nt = 4)

In [15]:
d = next(data_source)

In [17]:
d[1]

{258.0: array([[3.91454833e-08, 3.91455011e-08, 3.91455188e-08, 3.91455366e-08],
        [1.19109195e-07, 1.19109188e-07, 1.19109188e-07, 1.19109188e-07],
        [9.10890634e-08, 9.10890634e-08, 9.10890563e-08, 9.10890563e-08]]),
 259.0: array([[-2.28877788e-08, -2.28877557e-08, -2.28877344e-08,
         -2.28877113e-08],
        [ 1.28461252e-07,  1.28461252e-07,  1.28461252e-07,
          1.28461252e-07],
        [ 1.31640917e-07,  1.31640931e-07,  1.31640931e-07,
          1.31640931e-07]]),
 515.0: array([[-6.20332656e-08, -6.20332585e-08, -6.20332514e-08,
         -6.20332443e-08],
        [ 9.35205957e-09,  9.35206401e-09,  9.35206756e-09,
          9.35207112e-09],
        [ 4.05518641e-08,  4.05518712e-08,  4.05518783e-08,
          4.05518854e-08]]),
 260.0: array([[-1.05179332e-07, -1.05179325e-07, -1.05179318e-07,
         -1.05179311e-07],
        [-1.93402503e-08, -1.93402432e-08, -1.93402361e-08,
         -1.93402308e-08],
        [ 8.40349088e-08,  8.40349230e-08,  8.40

In [12]:
f.vis[1]['BASELINE']

259.0

In [13]:
def vectorised():
    a = np.zeros(2)
    a[:] = 2, 3
    return a

def looped():
    a = np.zeros(2)
    a[0] = 2
    a[1] = 3
    return a

In [ ]:
%timeit vectorised

In [ ]:
%timeit looped